In [1]:
query = "Was bedeutet das Bewertungsmerkmal Denken im Entgeltrahmentarifvertrag Baden-Württemberg?"

In [2]:
from langchain.document_loaders.pdf import PyMuPDFLoader

loader = PyMuPDFLoader('./data/Handbuch_der_ERA-Arbeitsbewertung.pdf')
docs = loader.load_and_split()
docs

[Document(page_content='Handbuch der ERA-Arbeitsbewertung \nErläuterungen und Glossar zur Anlage 1 \ndes ERA-TV \nfür die Beschäftigten in der Metall- und \nElektroindustrie in Baden-Württemberg (ERA-TV) \n \n \n \n \n  \n2023 – V2.8', metadata={'source': './data/Handbuch_der_ERA-Arbeitsbewertung.pdf', 'file_path': './data/Handbuch_der_ERA-Arbeitsbewertung.pdf', 'page': 0, 'total_pages': 270, 'format': 'PDF 1.5', 'title': 'Rechtssache', 'author': 'Behrle, Paula', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20230915164334+02'00'", 'modDate': "D:20230915164334+02'00'", 'trapped': ''}),
 Document(page_content='2 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nRedaktion: \nKatharina Hannemann \nStefan Orlowski \nWolfgang Pfeffer \nAndrea Schlotterer \nKai Schweppe \nChristine Trittel \n(Basierend auf den Arbeiten von Herbert Bickel) \n \nHinweis: \nZur besseren Lesbarkeit wird

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
vector_store = Chroma.from_documents(docs, embedding_function)
vector_store

/Users/tobiasoberrauch/Repositories/sandbox/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tobiasoberrauch/Repositories/sandbox/rag/.venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/tobiasoberrauch/Repositories/sandbox/rag/.venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
from langchain.llms.ollama import Ollama

llm = Ollama(model="mistral", temperature=0)

In [10]:
from langchain.prompts import PromptTemplate

refine_prompt_template = """
    Die ursprüngliche Frage lautet: \n {question} \n
    Die bereitgestellte Antwort lautet: \n {existing_answer}\n
    Verfeinern Sie die vorhandene Antwort bei Bedarf mit dem folgenden Kontext: \n {context_str} \n
    Erstellen Sie anhand des extrahierten Inhalts und der Frage eine endgültige Antwort.
    Wenn die Antwort nicht im Kontext enthalten ist, sagen Sie „Antwort im Kontext nicht verfügbar.“ \n\n
"""
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_prompt_template,
)


initial_question_prompt_template = """
    Beantworten Sie die Frage so präzise wie möglich und verwenden Sie ausschließlich den angegebenen Kontext. \n\n
    Kontext: \n {context_str} \n
    Frage: \n {question} \n
    Antwort:
"""

initial_question_prompt = PromptTemplate(
    input_variables=["context_str", "question"],
    template=initial_question_prompt_template,
)

In [11]:
from langchain.chains.question_answering import load_qa_chain

refine_chain = load_qa_chain(
    llm,
    chain_type="refine",
    return_intermediate_steps=True,
    question_prompt=initial_question_prompt,
    refine_prompt=refine_prompt,
)

In [12]:
refine_outputs = refine_chain({"input_documents": docs, "question": query})
refine_outputs

In [ ]:
from pathlib import Path

final_refine_data = []
for doc, out in zip(
    refine_outputs["input_documents"], refine_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = Path(doc.metadata["source"]).stem
    output["file_type"] = Path(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["answer"] = out
    final_refine_data.append(output)

In [ ]:
import pandas as pd

pdf_refine_answers = pd.DataFrame.from_dict(final_refine_data)
pdf_refine_answers = pdf_refine_answers.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_refine_answers.reset_index(inplace=True, drop=True)
pdf_refine_answers.head()

In [ ]:
pdf_refine_answers

In [ ]:
index = 0
print("[Context]")
print(pdf_refine_answers["chunks"].iloc[index])
print("\n\n [Answer]")
print(pdf_refine_answers["answer"].iloc[index])
print("\n\n [Page number]")
print(pdf_refine_answers["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_refine_answers["file_name"].iloc[index])